## Agents Demystified - part 2

## Introduction

In this tutorial we'll build on the results from [part 1](agents_pt1.ipynb) and explore letting the agent write and execute code and interact with data. In order to keep the notebook uncluttered, the code from part 1 is contained in a separate [file](helpers/agents2.py) that we'll import and use as-is.

So, without further ado, let's get started!

In [ ]:
!pip -q install ollama tavily

In [ ]:
import os

from helpers.agents2 import Agent
from helpers.agents2 import (date, calculator, web_search)

In [ ]:
# Host and model definitions
OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'gemma3:27b-it-qat'

os.environ['TAVILY_API_KEY'] = "" # <-- paste your key between the quotes before running this cell

## Baseline test

Just to make sure everything works as expected (i.e. where we left off in the previuos part):

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[date, web_search, calculator])
print(agent.task("what is the sum of the today's noon temperature in Paris and Berlin? Answer in centigrades. Is the answer reasonable given today's date?"))

In [ ]:
print(agent.message_history())

## Agent 007 with licence to code

As we saw before, asking our agent for the time without providing a helper tool likely results in a hallucination, or in the best case, a polite refusal to tell the time:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[])
print(agent.task("What time is it?"))

This time, we'll take a more roundabout way to solve the problem and let the agent code its own helper!

For this we need a generic tool to execute a python script<sup>1</sup> that the agent can call with the source code as argument.

---
<p><small>1. Allowing agents to execute code is obviously dangerous, and in a real application this tool would have to be sandboxed and locked down but as that is outside the scope of this tutorial no restrictions apply. You have been warned, be careful with what you ask the agent to do.</small></p>

---

In [ ]:
def execute_script(script: str) -> str:
    """
    Excecute python code and return the result as a string.
    You may import any python module, e.g. datetime or pandas
    If the script produce a figure, write it to a PNG file in the current working directory and return its name as a string using the format '## Figure: [name] ##' so it is visible to the user,

    Args:
        script (str): The python script to evaluate

    Returns:
        str: the result of running the script or an error message in case of failure

    """
    import subprocess
    import os
    import re

    from IPython.display import Image, display_png
    
    script_filename = "temp_script.py"
    work_dir = "work"
    output = ""
    
    with open(f"{work_dir}/{script_filename}", "w") as script_file:
        script_file.write(script)
    try:
        result = subprocess.run(
            ["python", script_filename],
            capture_output=True,
            cwd=work_dir,
            text=True,
            check=True,
        )
        output = result.stdout  
        # Uncomment next line to get debbuging output
        # print("Script output:", output) 
    except subprocess.CalledProcessError as e:
        print("Script execution failed:", e.stderr)  

    # Check if a figure was produced and if so, dispaly it
    RE_FIG = re.compile(r'## Figure:\s*(\S+)')
    fig_match = RE_FIG.match(output)
    if fig_match:
        fig_path = f"{work_dir}/{fig_match.group(1)}"
        display_png(Image(filename=fig_path))

    return output

This is a vanilla use of python's `subprocess` module to invoke python on a script (`temp_script.py`) containing the code written by the agent. If a an image was produced by the script (as per the docstring) it is rendered with the output.

In [ ]:
# Caveman test: check for syntax errors in execute_script(), should print 'Hello\n' unless there are errors
execute_script("print('Hello')")
# If calling execute_script() results in an error, restart the kernel (menu Kernel -> Restart Kernel…)
# and start over from the beginning

Now we can ask for the time again, this time providing `execute_script` as the only tool:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("What time is it?"))

In [ ]:
print(agent.message_history())

That's it!

Without any further changes, we can now give the agent complex queries, involving data analysis and visualiztion:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("You have permission to load the data file 'titanic.csv' in your default working directory. The file contains facts about the fate of the passengers abord the Titanic. Give me a verbal summary of the main conclusions that can be drawn from those facts."))

In [ ]:
# print(agent.message_history())

If we don't instantiate a new agent, we can continue to ask questions about the data …

In [ ]:
# same agent, continued...
print(agent.task("How many passengers were there, and how many survived?"))

… and we can ask for visualizations of data:

In [ ]:
# same agent, continued...
print(agent.task("Display a diagram showing survival versus fare"))

In [ ]:
# print(agent.message_history())

The above prompt was intentionally vague in the hope that the plot was not really an informative one (the actual result may vart from run to run). The good thing is that we can give the agent feedback in order to improve it:  

In [ ]:
# same agent, continued...
print(agent.task("Not quite what I wanted. Use likelihood of survival (in percent) versus fare, bin the fare into suitable ranges, and annotate the bars with the number of passengers in each bin."))

In [ ]:
print(agent.message_history())

## Final comments

With just a very simple tool (`execute_script`) we have created an agent that is a lot more useful than a plain LLM. In the final part we'll combine our agent with the WARA-ops portal data client. Stay tuned!

In the Appendix below a sample transcript (with a little formatting to make easier on the eyes) of the above interaction with the agent is given, just in case… 

### Appendix: Complete transcription of a successful conversation (for reference)

**user**:

You have permission to load the data file 'titanic.csv' in your default working directory. The file contains facts about the fate of the passengers abord the Titanic. Give me a verbal summary of the main conclusions that can be drawn from those facts.

**assistant**:

Thought: I need to load the 'titanic.csv' file and analyze its content to summarize the main conclusions about the fate of the passengers. I will use the `execute_script` tool to read the file and perform basic data analysis using pandas.  
Action: execute_script  
Action Input: `{"script": "import pandas as pd\n\ndf = pd.read_csv('titanic.csv')\n\nsurvived_percentage = (df['Survived'].sum() / len(df)) * 100\n\nprint(f'Overall survival rate: {survived_percentage:.2f}%')\n\ngender_survival = df.groupby('Sex')['Survived'].mean() * 100\nprint(f'Survival rate by gender:\\n{gender_survival}')\n\nclass_survival = df.groupby('Pclass')['Survived'].mean() * 100\nprint(f'Survival rate by passenger class:\\n{class_survival}')\n\nprint(f'Number of passengers in each class:\\n{df['Pclass'].value_counts()}')"}`


**user**:
```
Observation: Overall survival rate: 38.56%
Survival rate by gender:
Sex
female    74.203822
male      19.022688
Name: Survived, dtype: float64
Survival rate by passenger class:
Pclass
1    62.962963
2    47.282609
3    24.435318
Name: Survived, dtype: float64
Number of passengers in each class:
Pclass
3    487
1    216
2    184
Name: count, dtype: int64
```

**assistant**:

Thought: I have the data analysis results. I can now summarize the main conclusions.  
Action: answer  
Action Input: `{"reply": "Based on the analysis of the Titanic passenger data, several key conclusions can be drawn:\n\n1. **Overall Survival Rate:** Approximately 38.56% of the passengers survived the disaster.\n\n2. **Gender Disparity:** Female passengers had a significantly higher survival rate (74.20%) compared to male passengers (19.02%). This suggests a 'women and children first' protocol was in effect during the evacuation.\n\n3. **Passenger Class Impact:** Survival rates varied substantially by passenger class. First-class passengers (62.96%) had the highest survival rate, followed by second-class passengers (47.28%), with third-class passengers experiencing the lowest survival rate (24.44%).  This suggests a correlation between socioeconomic status and survival probability.\n\n4. **Number of Passengers:** The majority of passengers were in the third class (487), followed by first class (216), and then second class (184)."}`


**user**:

How many passengers were there, and how many survived?

**assistant**:

Thought: I can use the `execute_script` tool to load the file and count the total number of passengers and those who survived.  
Action: execute_script. 
Action Input: `{"script": "import pandas as pd\n\ndf = pd.read_csv('titanic.csv')\n\ntotal_passengers = len(df)\n\nsurvived_passengers = df['Survived'].sum()\n\nprint(f'Total number of passengers: {total_passengers}')\nprint(f'Number of survived passengers: {survived_passengers}')"}`


**user**:
```
Observation: Total number of passengers: 887
Number of survived passengers: 342
```

**assistant**:

Thought: I have the data. I can answer the question.  
Action: answer  
Action Input: `{"reply": "There were a total of 887 passengers on the Titanic, and 342 of them survived."}`


**user**:

Display a diagram showing survival versus fare

**assistant**:

Thought: I need to create a diagram showing the relationship between survival and fare. I'll use the `execute_script` tool with matplotlib to create a scatter plot and save it as a PNG file.  
Action: execute_script  
Action Input: `{"script": "import pandas as pd\nimport matplotlib.pyplot as plt\n\ndf = pd.read_csv('titanic.csv')\n\nplt.figure(figsize=(10, 6))\nplt.scatter(df['Fare'], df['Survived'], alpha=0.5)\nplt.xlabel('Fare')\nplt.ylabel('Survived')\nplt.title('Survival vs. Fare')\nplt.savefig('survival_vs_fare.png')\nprint('## Figure: survival_vs_fare.png ##')"}`


**user**:

Observation: ## Figure: survival_vs_fare.png ##


**assistant**:

Thought: I have successfully created and saved the diagram.  
Action: answer  
Action Input: `{"reply": "I have created a diagram showing the relationship between survival and fare. The diagram is saved as 'survival_vs_fare.png' in the current working directory."}`


**user**:

Not quite what I wanted. Use likelihood of survival (in percent) versus fare, bin the fare into suitable ranges, and annotate the bars with the number of passengers in each bin.

**assistant**:

Thought: I need to refine the diagram. I'll use the `execute_script` tool with pandas and matplotlib to bin the fare into ranges, calculate the average survival rate (likelihood) for each bin, and annotate the bars with the number of passengers in each bin.  
Action: execute_script  
Action Input: `{"script": "import pandas as pd\nimport matplotlib.pyplot as plt\n\ndf = pd.read_csv('titanic.csv')\n\n# Bin the fare into ranges\nbins = [0, 10, 20, 50, 100, 200, 500]\nfare_labels = ['0-10', '10-20', '20-50', '50-100', '100-200', '200+']\ndf['Fare_Bin'] = pd.cut(df['Fare'], bins=bins, labels=fare_labels)\n\n# Calculate the average survival rate for each bin\nfare_survival = df.groupby('Fare_Bin')['Survived'].mean() * 100\n\n# Calculate the number of passengers in each bin\nfare_counts = df['Fare_Bin'].value_counts().sort_index()\n\n# Create the bar plot\nplt.figure(figsize=(10, 6))\nplt.bar(fare_survival.index, fare_survival.values)\nplt.xlabel('Fare Range')\nplt.ylabel('Survival Rate (%)')\nplt.title('Survival Rate vs. Fare Range')\n\n# Annotate the bars with the number of passengers\nfor i, count in enumerate(fare_counts):\n    plt.text(i, fare_survival[i] + 1, str(count), ha='center')\n\nplt.savefig('survival_vs_fare_binned.png')\nprint('## Figure: survival_vs_fare_binned.png ##')"}`


**user**:

Observation: ## Figure: survival_vs_fare_binned.png ##


**assistant**:

Thought: I have successfully generated the binned diagram with annotations.  
Action: answer   
Action Input: `{"reply": "I have created a diagram showing the likelihood of survival (in percent) versus fare, with the fare binned into suitable ranges and each staple annotated with the number of passengers in each bin. The diagram is saved as 'survival_vs_fare_binned.png' in the current working directory."}`

And the final plot as rendered in this run:

![final plot](img/img1.png)